# Making test data for the test suite

**Goal:** Make a test bam file and a test vcf file for the test suite. And compsoe a json as SILO requires it after.

Here is the raw data we will use to create the test data:

In [ ]:
"""
Coor  12345678901234 *5678901234567890123456789012345
ref   AGCATGTTAGATAA**GATAGCTGTGCTAGTAGGCAGTCAGCGCCAT
              AAAAGA
+r001/1     TTAGATAAAGGATA*CTG
+r002      aaaAGATAA*GGATA
+r003    gcctaAGCTAA
+r004                  ATAGCT..............TCAGC
-r003                         ttagctTAGGC
-r001/2                                     CAGCGGCAT
"""

'\nCoor  12345678901234 5678901234567890123456789012345\nref   AGCATGTATAAAGATA*CGTCTGTAGCCCTCGCCCAT\n\n+r001/1  TTAGATAAAGGATA*CTG\n+r002     aaaAGATAA*GGATA\n+r003     gcctAAGCTAA\n+r004              ATAGCT...........TCAGC\n-r003                    ttagctTAGGC\n-r001/2                           CAGCCGCAT\n'

In [ ]:
"""
@HD VN:1.6 SO:coordinate
@SQ SN:ref LN:45
r001    99 ref  7 30 8M2I4M1D3M * 37  39 TTAGATAAAGGATACTG *
r002     0 ref  9 30 3S6M1P1I4M *  0   0 AAAAGATAAGGATA     * SA:Z:ref,29,-,6H5M,17,0;
r003     0 ref  9 30 5S6M       *  0   0 GCCTAAGCTAA        *
r004  2064 ref 16 30 6M14N5M    *  0   0 ATAGCTTCAGC        *
r003  2064 ref 29 17 6H5M       *  0   0 TAGGC              * SA:Z:ref,9,+,5S6M,30,1;
r001   147 ref 37 30 9M         *  7 -39 CAGCGGCAT          * NM:i:1
"""

'\n@HD VN:1.6 SO:coordinate\n@SQ SN:ref LN:45\nr001 99 ref 7 30 8M2I4M1D3M * 37 39 TTAGATAAAGGATA*CTG *\nr002 0 ref 9 30 3S6M1P1I4M * 0 0 AAAAGATAA*GGATA * SA:Z:ref,29,-,6H5M,17,0;\nr003 0 ref 9 30 5S6M * 0 0 GCCTAAGCTAA *\nr004 2064 ref 16 30 6M14N5M * 0 0 ATAGCTTCAGC *\nr003 2064 ref 29 17 6H5M * 0 0 TAGGC * SA:Z:ref,9,+,5S6M,30,1;\nr001 147 ref 37 30 9M * 7 -39 CAGCCGCAT * NM:i:1\n'

In [4]:
import pysam

In [78]:
# lets create a sam file out of the above
with open("test.sam", "w") as f:
    f.write(
        """@HD VN:1.6 SO:coordinate
@SQ SN:ref LN:45
r001    99  ref 7 30    8M2I4M1D3M = 37  39 TTAGATAAAGGATACTG   *
r002    0   ref 9 30    3S6M1P1I4M *  0   0 AAAAGATAAGGATA  * SA:Z:ref,29,-,6H5M,17,0;
r003    0   ref 9 30    5S6M       *  0   0 GCCTAAGCTAA *
r004    2064    ref 16  30 6M14N5M    *  0   0 ATAGCTTCAGC  *
r003    2064    ref 29  17 6H5M       *  0   0 TAGGC    * SA:Z:ref,9,+,5S6M,30,1;
r001    147 ref 37  30 9M         =  7 -39 CAGCGGCAT    * NM:i:1
"""
    )

tmpfilename = "test.bam"

header = {
    "HD": {"VN": "1.6", "SO": "coordinate"},
    "SQ": [{"SN": "ref", "LN": 45}],
}

with pysam.AlignmentFile(tmpfilename, "wb", header=header) as outf:
    a = pysam.AlignedSegment()
    a.query_name = "r001"
    a.query_sequence = "TTAGATAAAGGATACTG"
    a.flag = 99
    a.reference_id = 0
    a.reference_start = 7
    a.mapping_quality = 30
    a.cigar = [(0, 8), (1, 2), (0, 4), (2, 1), (0, 3)]  # Fixed CIGAR: 8M2I4M1D3M
    a.next_reference_id = 0
    a.next_reference_start = 37
    a.template_length = 39
    a.query_qualities = [0] * 17
    a.tags = []
    outf.write(a)

    b = pysam.AlignedSegment()
    b.query_name = "r002"
    b.query_sequence = "AAAAGATAAGGATA"
    b.flag = 0
    b.reference_id = 0
    b.reference_start = 9
    b.mapping_quality = 30
    b.cigar = [(4, 3), (0, 6), (1, 1), (4, 4)]
    b.next_reference_id = 0
    b.next_reference_start = 0
    b.template_length = 0
    b.query_qualities = [0] * 14
    outf.write(b)

    c = pysam.AlignedSegment()
    c.query_name = "r003"
    c.query_sequence = "GCCTAAGCTAA"
    c.flag = 0
    c.reference_id = 0
    c.reference_start = 9
    c.mapping_quality = 30
    c.cigar = [(4, 5), (0, 6)]
    c.next_reference_id = 0
    c.next_reference_start = 0
    c.template_length = 0
    c.query_qualities = [0] * 11
    outf.write(c)

    d = pysam.AlignedSegment()
    d.query_name = "r004"
    d.query_sequence = "ATAGCTTCAGC"
    d.flag = 2064
    d.reference_id = 0
    d.reference_start = 16
    d.mapping_quality = 30
    d.cigar = [(0, 6), (3, 14), (0, 5)]
    d.next_reference_id = 0
    d.next_reference_start = 0
    d.template_length = 0
    d.query_qualities = [0] * 11
    outf.write(d)

    e = pysam.AlignedSegment()
    e.query_name = "r003"
    e.query_sequence = "TAGGC"
    e.flag = 2064
    e.reference_id = 0
    e.reference_start = 29
    e.mapping_quality = 17
    e.cigar = [(5, 6), (0, 5)]
    e.next_reference_id = 0
    e.next_reference_start = 0
    e.template_length = 0
    e.query_qualities = [0] * 5
    e.tags = [("SA", "Z:ref,9,+,5S6M,30,1")]
    outf.write(e)

    f = pysam.AlignedSegment()
    f.query_name = "r001"
    f.query_sequence = "CAGCGGCAT"
    f.flag = 147
    f.reference_id = 0
    f.reference_start = 37
    f.mapping_quality = 30
    f.cigar = [(0, 9)]
    f.next_reference_id = 0
    f.next_reference_start = 7
    f.template_length = -39
    f.query_qualities = [0] * 9
    f.tags = [("NM", "i:1")]
    outf.write(f)

# close the file
outf.close()





In [79]:
index = pysam.index(tmpfilename)

In [63]:
# read in bam and print out the reads
with pysam.AlignmentFile(tmpfilename, "rb") as f:
    for read in f.fetch():
        print(
            read.query_name,
            read.query_sequence,
            read.query_qualities,
            read.flag,
            read.reference_id,
            read.reference_start,
            read.mapping_quality,
            read.cigarstring,
            read.next_reference_id,
            read.next_reference_start,
            read.template_length,
        )
        print("tags:", read.tags)
        print("query_qualities:", read.query_qualities)
        print("query_length:", read.query_length)
        print("query_alignment_length:", read.query_alignment_length)
        print("query_alignment_start:", read.query_alignment_start)
        print("query_alignment_end:", read.query_alignment_end)
        print("reference_length:", read.reference_length)
        print("reference_start:", read.reference_start)
        print("reference_end:", read.reference_end)



r001 TTAGATAAAGGATACTG array('B', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) 99 0 7 30 8M2I4M1D3M 0 37 39
tags: []
query_qualities: array('B', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
query_length: 17
query_alignment_length: 17
query_alignment_start: 0
query_alignment_end: 17
reference_length: 16
reference_start: 7
reference_end: 23
r002 AAAAGATAAGGATA array('B', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) 0 0 9 30 3S6M1I4S 0 0 0
tags: []
query_qualities: array('B', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
query_length: 14
query_alignment_length: 7
query_alignment_start: 3
query_alignment_end: 10
reference_length: 6
reference_start: 9
reference_end: 15
